**Thsi file contains code for the entire pipeline for deployment purpose using flask**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/my_test.zip

In [ ]:
pip install flask_ngrok

In [ ]:
from flask_ngrok import run_with_ngrok

In [ ]:
import pickle
from flask import Flask, jsonify, request
import flask

In [ ]:
import os
from PIL import Image
from tqdm import tqdm
import numpy as np
import pandas as pd
import shutil
import cv2
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
meta_test=pd.read_csv('/content/drive/MyDrive/meta_test.csv')

In [ ]:
meta_test_sorted=meta_test.sort_values(by='image_id')

**Cloning the yolo-v5 repository**

In [ ]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 9909, done.
remote: Total 9909 (delta 0), reused 0 (delta 0), pack-reused 9909
Receiving objects: 100% (9909/9909), 10.06 MiB | 9.98 MiB/s, done.
Resolving deltas: 100% (6899/6899), done.


**Inferencing by using yolo-v5 weights**

In [ ]:
!python /content/yolov5/detect.py --weights '/content/drive/MyDrive/best.pt' '/content/drive/MyDrive/best (1).pt' '/content/drive/MyDrive/best (3).pt' '/content/drive/MyDrive/best (4).pt' '/content/drive/MyDrive/best (5).pt' --source /content/resized_test_data  --img 256  --iou-thres 0.5 --conf 0.2 --max-det 5 --save-txt --save-conf

detect: weights=['/content/drive/MyDrive/best.pt', '/content/drive/MyDrive/best (1).pt', '/content/drive/MyDrive/best (3).pt', '/content/drive/MyDrive/best (4).pt', '/content/drive/MyDrive/best (5).pt'], source=/content/resized_test_data, imgsz=[256, 256], conf_thres=0.2, iou_thres=0.5, max_det=5, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13

requirements: 1 package updated per /content/yolov5/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 v6.0-30

In [ ]:
app=Flask(__name__,template_folder='/content/templates')

In [ ]:
picFolder = os.path.join('static', 'pics')

In [ ]:
app.config['UPLOAD_FOLDER'] = picFolder

In [ ]:
run_with_ngrok(app)

In [ ]:
@app.route('/')
def index():
  return flask.render_template('index.html')

**This is the entire pipeline which processes the image file given by the user and generates the predictions**

In [ ]:
@app.route('/predict',methods=['POST'])
def predict():
  file=request.form.to_dict()
  file_name=file['Uploaded_Image']
  file_location='/content/resized_test_data/'+file_name
  img_level_pred={}
  img_level_predictions=[]
  img_level_id=[]
  for img_test in [file_name]:
    img_level_readings=[]
    for img_txt in os.listdir('/content/yolov5/runs/detect/exp/labels'):
      if img_txt.split('.')[0]==img_test.split('.')[0]:
        dim_0= meta_test_sorted[meta_test_sorted['image_id']==img_txt.split('.')[0]]['dim0']
        dim_1= meta_test_sorted[meta_test_sorted['image_id']==img_txt.split('.')[0]]['dim1']

        scale_fact_x=dim_1/256
        scale_fact_y=dim_0/256

        with open('/content/yolov5/runs/detect/exp/labels/'+ img_txt) as f:
          for line in f:
            b=line.strip('\n').split(' ')
            bb_conf_coord=[]
            for num in b:
              bb_conf_coord.append(eval(num))
            conf_box=bb_conf_coord[5]
            xc,yc=int(bb_conf_coord[1]*256),int(bb_conf_coord[2]*256)
            w,h=int(bb_conf_coord[3]*256),int(bb_conf_coord[4]*256)
    
            xmin = xc - (w/2)
            ymin = yc - (h/2)
            xmax = xc + (w/2)
            ymax = yc + (h/2)

            xmin=int(xmin)
            xmax=int(xmax)
            ymin=int(ymin)
            ymax=int(ymax)

          
            img_level_readings.append([conf_box,xmin,ymin,xmax,ymax])
    if len(img_level_readings)==0:

      img_level_readings.append([])

  
    img_level_predictions.append(img_level_readings)
    img_level_id.append(img_test.split('.')[0]+'_image')
  img_level_pred.update({'id':img_level_id,'PredictionString':img_level_predictions})
  img_level_pred_df=pd.DataFrame(img_level_pred)
  top_model=load_model('/content/drive/MyDrive/top_model.h5')
  test_datagen=ImageDataGenerator(rescale=1/255)
  test_path=[]
  test_path.append('/content/resized_test_data/'+file_name)
  test_df=pd.DataFrame({'path':test_path})
  test_generator=test_datagen.flow_from_dataframe(dataframe=test_df,x_col='path',target_size=(256,256),batch_size=1,class_mode=None,shuffle=False)
  prediction=top_model.predict(test_generator)
  test_prediction=np.argmax(prediction,axis=1)
  for ind in img_level_pred_df.index:
    if len(img_level_pred_df['PredictionString'][ind][0])>0:
      img_name=img_level_pred_df['id'][ind].replace('_image','.jpg')
      img=np.array(Image.open('resized_test_data/'+img_name))
      for j in range(len(img_level_pred_df['PredictionString'][ind])):
        image=cv2.rectangle(img,(int(img_level_pred_df['PredictionString'][ind][j][1]),int(img_level_pred_df['PredictionString'][ind][j][2]),int(img_level_pred_df['PredictionString'][ind][j][3]),int(img_level_pred_df['PredictionString'][ind][j][4])),(0,255,0),2)
        cv2.putText(image,'opacity '+str(np.round(img_level_pred_df['PredictionString'][ind][j][0],2)),(int(img_level_pred_df['PredictionString'][ind][j][1]),int(img_level_pred_df['PredictionString'][ind][j][2])),cv2.FONT_HERSHEY_SIMPLEX,0.5,color=(255,0,0),thickness=1)
        if test_prediction[0]==0:
          cv2.putText(image,'negative',(int(img_level_pred_df['PredictionString'][ind][j][3]-50),int(img_level_pred_df['PredictionString'][ind][j][4]-50)),cv2.FONT_HERSHEY_SIMPLEX,0.4,color=(255,255,255),thickness=1)
        elif test_prediction[0]==1:
          cv2.putText(image,'typical',(int(img_level_pred_df['PredictionString'][ind][j][3]-50),int(img_level_pred_df['PredictionString'][ind][j][4]-50)),cv2.FONT_HERSHEY_SIMPLEX,0.4,color=(255,255,255),thickness=1) 
        elif test_prediction[0]==2:
          cv2.putText(image,'indeterminate',(int(img_level_pred_df['PredictionString'][ind][j][3]-50),int(img_level_pred_df['PredictionString'][ind][j][4]-50)),cv2.FONT_HERSHEY_SIMPLEX,0.4,color=(255,255,255),thickness=1)
        elif test_prediction[0]==3:
          cv2.putText(image,'atypical',(int(img_level_pred_df['PredictionString'][ind][j][3]-50),int(img_level_pred_df['PredictionString'][ind][j][4]-50)),cv2.FONT_HERSHEY_SIMPLEX,0.4,color=(255,255,255),thickness=1)
      
      out_image=Image.fromarray(image)
      out_image.save('/content/static/pics/'+file_name)
      out_image_path=os.path.join(app.config['UPLOAD_FOLDER'],file_name)
      return flask.render_template('index_modified.html',user_image=out_image_path)
     

    else:
      img_name=img_level_pred_df['id'][ind].replace('_image','.jpg')
      img=np.array(Image.open('resized_test_data/'+img_name))
      out_image=Image.fromarray(img)
      out_image.save('/content/static/pics/'+file_name)
      out_image_path=os.path.join(app.config['UPLOAD_FOLDER'],file_name)
      return flask.render_template('index_modified.html',user_image=out_image_path)

if __name__== '__main__':
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://2937-35-201-184-184.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [24/Oct/2021 10:52:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2021 10:52:19] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [24/Oct/2021 10:52:20] "GET / HTTP/1.1" 200 -


Found 1 validated image filenames.


127.0.0.1 - - [24/Oct/2021 10:52:29] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2021 10:52:29] "GET /static/pics/0a0c1312be06.jpg HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2021 10:52:30] "GET /predict HTTP/1.1" 405 -
